In [7]:
from pynq import Overlay
from pynq import allocate

In [3]:
# Load Overlay
overlay = Overlay('./cnna.bit')

In [5]:
overlay?

In [6]:
test_ip = overlay.mult_acc

In [14]:
in1 = allocate(shape=(9,), dtype='u2')